In [ ]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from util.data_generate import *
from util.methods import *
import csv
#worst
p = 50
betas = ['smallest','cluster','dynamic']
seed, J, n_iter, num_repeats = 1, 10000, 150, 30
np.random.seed(seed=seed)
Sigma1 = np.loadtxt('Data_generate/Sigma1.txt')
Sigma1 = Sigma1 * 4

Sigma2 = np.loadtxt('Data_generate/group_cov.txt')
std2 = np.sqrt(np.diag(Sigma2))
R2 = Sigma2 / np.outer(std2, std2)
mu = np.loadtxt('Data_generate/mu.txt')

for m in [16,]:
    U3, ids3 = SAA(p, m, mu, std2, R2, Sigma2, J=J)
    z_samples = np.random.randn(J*10, m)
    G_real = np.mean(np.min(mu[ids3] + (U3 @ z_samples.T).T, axis=1))
    cluster_labels = cluster_subjects(R2, n_clusters=m, method='average')
    for beta in betas:

        csv_path = f'{p}_{m}_{beta}.csv'
        csv_path1 = f'{p}_{m}_{beta}_pre.csv'

        # Initialize CSV
        with open(csv_path, 'w') as f:
            f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter+1)]) + '\n')
        with open(csv_path1, 'w') as f:
            f.write(','.join(['Repetition'] + [f'Step_{i}' for i in range(1, n_iter+1)]) + '\n')

        for repeat in tqdm(range(num_repeats)):
            # Initial setup
            seed = seed + repeat
            
            # Initialize variables
            mu_0 = np.full(p, 70)
            mu_hat, var_hat, obs_num = np.zeros(p), np.ones(p), np.zeros(p)
            
            # Initial observation
            N_0 = math.ceil(p / m)
            for i in range(N_0):
                ids = efficient_initialization(p, m, obs_num, mu_0, std2, R2, Sigma2, J)
                obs_num[ids] += 1
                indices = ids.copy()
                obs = np.random.multivariate_normal(mu[ids], Sigma2[np.ix_(ids, ids)])
                X_n = ids.copy()
                
                # Update estimates
                Gamma = Sigma2[np.ix_(ids, ids)]
                inv_mat = np.linalg.inv(Sigma1[np.ix_(ids, ids)] + Gamma)
                delta = (obs - mu_0[ids]).reshape(-1, 1)
                
                COV_x = Sigma1[:, ids]
                adjustments = (COV_x @ inv_mat @ delta).flatten()
                mu_hat = mu_0 + adjustments
                adjustments = np.diag(COV_x @ inv_mat @ COV_x.T)
                var_hat = np.diag(Sigma1) - adjustments
                
            # Main loop
            G = [] #用于计算累积遗憾
            G_pre = [] 
            for t in range(N_0+1, n_iter+1):
                if beta == 'dynamic':
                    dynamic_beta = np.sqrt(6 * np.log(t) / np.maximum(obs_num, 1))
                    U, ids = improvement(p, m, mu_hat - dynamic_beta * np.sqrt(var_hat), std2, R2, Sigma2, J=J)
                elif beta == 'cluster': 
                    ids = find_min_mu_in_clusters(mu_hat, cluster_labels)
                    sub_Sigma_cluster = Sigma2[np.ix_(ids, ids)]
                    U = np.linalg.cholesky(sub_Sigma_cluster) 
                else:
                    ids = np.argsort(mu_hat)[:m]
                    sub_Sigma_smallest = Sigma2[np.ix_(ids, ids)]
                    U = np.linalg.cholesky(sub_Sigma_smallest) 
                obs_num[ids] += 1
                indices = np.vstack((indices, ids))
                
                newobs = np.random.multivariate_normal(mu[ids], Sigma2[np.ix_(ids, ids)])
                X_n, obs = update_subject_stats(obs_num, X_n, obs, ids, newobs)
                
                # Update Gamma matrix
                len_Xn = len(X_n)
                Gamma = np.zeros((len_Xn, len_Xn))
                for i in range(len_Xn):
                    for j in range(i, len_Xn):
                        count = np.sum((indices == X_n[i]).any(1) & (indices == X_n[j]).any(1))
                        Gamma[i,j] = Sigma2[X_n[i],X_n[j]] * count / (obs_num[X_n[i]] * obs_num[X_n[j]])
                Gamma += Gamma.T
                np.fill_diagonal(Gamma, np.diag(Gamma)/2)
                
                # Update estimates
                COV_XX = Sigma1[np.ix_(X_n, X_n)]
                inv_mat = np.linalg.inv(COV_XX + Gamma)
                delta = (obs - mu_0[X_n]).reshape(-1, 1)
                
                COV_x = Sigma1[:, X_n]
                adjustments = (COV_x @ inv_mat @ delta).flatten()
                mu_hat = mu_0 + adjustments
                adjustments = np.diag(COV_x @ inv_mat @ COV_x.T)
                var_hat = np.diag(Sigma1) - adjustments

                # Calculate G
                z_samples = np.random.randn(J*10, m)
                G.append(np.mean(np.min(mu[ids] + (U @ z_samples.T).T, axis=1)))

                Gmin = 1000
                idmin = []
                z_samples2 = np.random.randn(J, m)
                for id in indices:
                    sub_Sigma = Sigma2[np.ix_(id, id)]
                    B = np.linalg.cholesky(sub_Sigma)
                    Gtemp = np.mean(np.min(mu_hat[id] + (B @ z_samples2.T).T, axis=1))
                    if Gtemp < Gmin:
                        idmin = id.copy()
                        Gmin = Gtemp
                ids1 = idmin

                sub_Sigma = Sigma2[np.ix_(ids1, ids1)]
                U1 = np.linalg.cholesky(sub_Sigma)
                G_pre.append(np.mean(np.min(mu[ids1] + (U1 @ z_samples.T).T, axis=1)))
            
            # Save results
            with open(csv_path, 'a') as f:
                f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G)-G_real]) + '\n')
            with open(csv_path1, 'a') as f:
                f.write(','.join([str(repeat)] + [f'{x:.6f}' for x in np.array(G_pre)-G_real]) + '\n')

100%|██████████| 30/30 [07:14<00:00, 14.49s/it]


In [3]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal
from util.data_generate import *
from util.methods import *
import csv
# 你的具体例子
ids1 = [1, 2, 3, 4, 5, 6, 7, 8]
ids2 = [1, 2, 3, 4, 5, 6, 7, 11]  # 只有最后一个元素不同
Sigma2 = np.loadtxt('Data_generate/group_cov.txt')

# 提取子矩阵
A = Sigma2[np.ix_(ids1, ids1)]
B = Sigma2[np.ix_(ids2, ids2)]

# 计算旧的Cholesky分解
U1 = np.linalg.cholesky(A)

# 正确的方法：从变化位置开始更新所有后续行
U2_corrected = cholesky_update(U1, B)

# 验证
U2_full = np.linalg.cholesky(B)

print("直接计算的U2:")
print(U2_full)
print("\n修正后的更新结果:")
print(U2_corrected)
print("\n两者是否相同?", np.allclose(U2_full, U2_corrected, atol=1e-10))

直接计算的U2:
[[10.48826473  0.          0.          0.          0.          0.
   0.          0.        ]
 [ 4.92991246  9.71229211  0.          0.          0.          0.
   0.          0.        ]
 [ 5.57206626  3.49414636  8.3294753   0.          0.          0.
   0.          0.        ]
 [ 5.37311514  2.89299131  0.35515183  7.85787109  0.          0.
   0.          0.        ]
 [ 5.14522892  3.64402001  1.14462796  1.70193345  7.99156561  0.
   0.          0.        ]
 [ 4.53690112  1.86199884  3.52416219  2.06202082 -1.06996252  7.02231399
   0.          0.        ]
 [ 3.69404921  1.32876376  2.88244327  1.84944091  2.20537299  1.92655792
   7.25198793  0.        ]
 [-0.2192213   0.67137229  0.4332055  -0.77437802  0.419542    0.2004193
   0.84546034  8.59466703]]

修正后的更新结果:
[[10.48826473  0.          0.          0.          0.          0.
   0.          0.        ]
 [ 4.92991246  9.71229211  0.          0.          0.          0.
   0.          0.        ]
 [ 5.57206626  3.49414636 